In [1]:
import sys
import os
sys.path.append("../../../../src/")
#Imports
import numpy as np
import scipy as sp
import numpy.linalg as la
import matplotlib.pyplot as plt
#Custom made imports
import single_particle_sector as sps
from time import time


In [2]:
###GENERAL SIGMA X X 
def remove_duplicates_in_pairs(vec):
    unique_vals, counts = np.unique(vec, return_counts=True)
    filtered_vals = unique_vals[counts % 2 != 0]
    return filtered_vals.tolist()

In [3]:
indices = np.array([0,9,15,30])

L = 100
J = -1
h = 0.5


####CALCULATE OLD WAY
E, V = la.eigh(sps.H_bdg(h,L,J))
V = V[:,:L]
G = V @ V.conj().T
timer = time()

DAT  = sps.sigma_general(indices,G)
print(f"sigma {indices} = {DAT}")
TIME1 = time()-timer
print(f"computed in {TIME1} sec")



TypeError: sigma_general() missing 1 required positional argument: 'L'

In [4]:

#Caclulate NEW WAY
timer = time()
E, V = la.eigh(sps.H_bdg(h,L,J))
V = V[:,:L]
G = V @ V.conj().T
F = G[:L,L:]
G = G[:L,:L]
M = np.eye(L)- 2*(G+F)
#Should only need indices and correlation matrix to calculate


#### Function does everything below
#Sort Indices and remove duplicates. Always possible due to behaviour of pauli spin matrices
timer2 = time()
indices = np.sort(indices)
indices = remove_duplicates_in_pairs(indices)

# Get pairs of sites
odd_sites = np.array(indices[::2])
even_sites= np.array(indices[1::2])
JW_string_lengths = even_sites-odd_sites
N = sum(JW_string_lengths)

R = []
for i in range(0, len(indices), 2):
    start = indices[i]
    end = indices[i+1]
    R.extend(range(start, end+1))

A_coords = [x for x in R if x not in odd_sites]
B_coords = [x for x in R if x not in even_sites]

###Building C
C = np.zeros( (N,N))
for nx in range(N):
    for ny in range(N):
        Bx = B_coords[nx]
        Ay = A_coords[ny]

        C[nx,ny] = M[Bx,Ay]

DAT2 = la.det(C)
TIME2 = time()-timer
TIME3 = time()-timer2
print(f"Sigma {indices} = {DAT2}")
print(f"Time = {TIME2}")
print(f"Delta = {DAT-DAT2}")
print(f"time save ={TIME1-TIME2}")
print(f"no eig time save= {TIME1-TIME3} a which is {(TIME1-TIME3)/TIME1}% faster")

Sigma [0, 9, 15, 30] = 0.8660256849028556
Time = 0.026569128036499023


NameError: name 'DAT' is not defined

Holy shit it's 99% faster this way?!?!?

In [5]:
##Packaging
import numpy as np

def random_integer_strings(num_strings, L):
    strings = []
    for _ in range(num_strings):
        length = np.random.randint(1, L+1)
        string = np.random.randint(1, L+1, size=length).tolist()
        strings.append(string)
    return strings


def sigma_general(indices,Gi):
    """
    Calculates the expectation values of sigma_x operators put at arbitary sites"
    Inputs:
    indices = list of integers
    G = 2L x2L correlation matrix corresponding to quantum state of interest

    Outputs:
    complex scalar 

    Example:

    indices = [1, 2, 3, 4, 5] 
    sigma_5pt = sigma_general(indices,G)
    """
    #Correlation Matrices
    F = Gi[:L,L:]
    G = Gi[:L,:L]
    #M[x,y] = <BxAy>
    M = np.eye(L)- 2*(G+F)
    #Sigma matrices on different sites commute
    indices = np.sort(indices)
    #Remove any duplicates as sigma_x^2 = 1
    indices = remove_duplicates_in_pairs(indices)
    if len(indices)%2 == 1:
        constant = 10
        indices = list(indices) + [x + constant for x in indices]
        return np.sqrt(np.abs(sigma_general(indices,Gi)))
    
    #Bs sit on odd sites
    odd_sites = np.array(indices[::2])
    #As site on even sites
    even_sites= np.array(indices[1::2])
    #Get string lengths
    JW_string_lengths = even_sites-odd_sites\
    #Sum of string lengths is size of matrix needed
    N = sum(JW_string_lengths)
    #Fill in indices for strings
    R = []
    for i in range(0, len(indices), 2):
        start = indices[i]
        end = indices[i+1]
        R.extend(range(start, end+1))

    A_coords = [x for x in R if x not in odd_sites]
    B_coords = [x for x in R if x not in even_sites]

    ###Building C
    C = np.zeros( (N,N))
    for nx in range(N):
        for ny in range(N):
            Bx = B_coords[nx]
            Ay = A_coords[ny]

            C[nx,ny] = M[Bx,Ay]

    return la.det(C)

In [6]:
L = 100
J = 1
h = 0
E, V = la.eigh(sps.H_bdg(h,L,J))
V = V[:,:L]
G = V @ V.conj().T
indices=[1,2]
timer = time()

timer = time()
print(sps.sigma_general(indices,G,L))
print(time()-timer)


-1.0000000000000002
0.00024700164794921875


In [7]:
n = np.array([i for i in range(14)])
print(n)
times = []
for ni in n:
    timer = time()
    sps.P_n(ni,G,L)
    times.append(time()-timer)
times = np.array(times)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


In [8]:
times[0]*2**(14)

np.float64(3.37890625)

In [9]:

# Example usage:
MAX = 10
num_strings = 5
random_strings = random_integer_strings(num_strings, MAX)
print(random_strings)


[[9, 8, 4, 6, 1, 2, 6, 10, 9], [9, 6, 2, 5, 6, 6, 2, 10, 7], [2, 9, 3], [6, 9, 5, 4, 6], [5, 6, 3, 1]]


In [10]:
#Initialize Test
J = 1
L = 150
h_i = np.linspace(0,2,100)

dat = []
dat2= []
dat3= []
for h in h_i:
    H = sps.H_bdg(h,L,J)
    E,V = la.eigh(H)
    G = sps.G_tfim(V[:,:L])
    dat.append(sps.P_n(2,G,L))
    dat2.append(sps.P_n(4,G,L))
    #dat3.append(sps.P_n(8,G))
plt.plot(h_i,dat, label = "P_n(2)")
plt.plot(h_i,dat2, label = "P_n(4)")
#plt.plot(h_i,dat3, label = "P_n(8)")
plt.legend()
plt.xlabel("h")
plt.ylabel("P_n")
plt.vlines(1,0,.5, color = "black", linestyle = "--")
plt.xlim(h_i[0],h_i[-1])
plt.ylim(0,1)

NameError: name 'x' is not defined